# Ticks Per Inch with Test Data

In [ ]:
import psycopg2 as pg
import pandas as pd
%matplotlib notebook

In [ ]:
connection = pg.connect("dbname=jeff user=jeff")

In [ ]:
query = """
SELECT id, name, timestamp, CAST(meta->>'direction' AS real) AS direction,
meta->>'robot' AS robot, meta->>'wheel' AS wheel
FROM tc_activity
"""

activity_info = pd.read_sql(query, index_col='id', con=connection)


In [ ]:
query = """
SELECT id, name, timestamp, measure, value
FROM tc_activity, unnest(activity_measures, data) AS u(measure, value);
"""

activity_data = pd.read_sql(query, con=connection)


In [ ]:
pivot = activity_data.pivot(index='id', columns='measure', values='value')
activity = pd.merge(activity_info, pivot, on='id')
activity['tpi'] = activity['actual_ticks'] / activity['actual_distance']
activity.groupby('direction').size()

In [ ]:
activity.loc[activity['actual_distance'] != 0].boxplot(column='tpi', by=['name', 'direction'],
                                                       rot=15, figsize=(7.5, 5))

In [ ]:
activity.loc[activity['direction'] == 180]['actual_distance']